In [1]:
import os

In [2]:
%pwd

'/Users/palakprashant/E2E_ML_Project_MLflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/palakprashant/E2E_ML_Project_MLflow'

In [5]:
import pandas as pd

In [6]:
data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [8]:
#EDA
#Check for missing values:
data.isnull().sum() #no missing values

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [9]:
data.shape

(1599, 12)

In [10]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [11]:
#Prepare the entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataValidationConfiguration:
    root_directory: Path
    status_file: str
    unzip_data_directory: Path
    entire_schema: dict

In [12]:
#Prepare Configuration Manager
from ml_proj.constants import *
from ml_proj.utils.common import read_yaml, create_directories

In [58]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = config_file,
            params_filepath = params_file,
            schema_filepath = schema_file):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_source])

    def get_data_validation_configuration(self) -> DataValidationConfiguration:
        configuration = self.config.data_validation
        schema = self.schema.COLUMNS
        create_directories[(configuration.root_directory)]

        data_validation_configuration = DataValidationConfiguration(
            root_directory = configuration.root_directory,
            status_file_val = configuration.status_file,
            unzip_data_directory_val = configuration.unzip_data_directory,
            entire_schema= schema,
        )
        return data_validation_configuration
        
    


In [59]:
import os
from ml_proj import logger

In [60]:
class DataValidation:
    def __init__(self, config: DataValidationConfiguration):
        self.config = config
    
    def validate_columns(self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_directory_val)
            all_columns = list(data.columns) 

            entire_schema = self.config.entire_schema.keys()

            for col in all_columns: #Compare with columns in the schema
                if col not in entire_schema:
                    validation_status = False
                    with open(self.config.status_file, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.status_file, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
            return validation_status
        except Exception as e:
            raise e
        
    

In [61]:
  #Update pipeline
try:
    config = ConfigurationManager()
    data_validation_configuration =  config.get_data_validation_configuration()
    data_validation = DataValidation(config=data_validation_configuration)
    print("this is fine")
    data_validation.validate_columns()
    print("this is fine")
except Exception as e:
    raise e

[2024-11-07 11:49:30,863: INFO: common: yaml file: src/ml_proj/config/config.yaml loaded successfully.]
[2024-11-07 11:49:30,865: INFO: common: yaml file: params.yaml loaded successfully.]
[2024-11-07 11:49:30,868: INFO: common: yaml file: schema.yaml loaded successfully.]
[2024-11-07 11:49:30,870: INFO: common: Created directory here: artifacts]


TypeError: 'WrappedFunction' object is not subscriptable